In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time

In [ ]:
img1 = cv2.imread('8.jpg', cv2.IMREAD_GRAYSCALE)  
img2 = cv2.imread('9.jpg', cv2.IMREAD_GRAYSCALE)  

In [ ]:
img1_cpy=cv2.imread('8.jpg', cv2.IMREAD_COLOR);
img2_cpy=cv2.imread('9.jpg', cv2.IMREAD_COLOR);

In [ ]:
if img1 is None or img2 is None:
    print("fail")
    exit()

In [ ]:
t1 = time.time()
time_sum=0;

In [ ]:
h1, w1 = img1.shape

In [ ]:
img2 = cv2.resize(img2, (w1, h1))

In [ ]:
h11,w11=img1_cpy.shape[:2]
img2_cpy=cv2.resize(img2_cpy,(w11,h11));

In [ ]:
print(1)

In [ ]:
sift = cv2.SIFT_create()

In [ ]:
kp1, des1 = sift.detectAndCompute(img1, None)  
kp2, des2 = sift.detectAndCompute(img2, None)  
print(2);

In [ ]:
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
matches = bf.knnMatch(des1, des2, k=2)  
print(3)

In [ ]:
good_matches = []
ratio_thresh = 0.75  
for m, n in matches:
    if m.distance < ratio_thresh * n.distance:
        good_matches.append(m)

In [ ]:
print(4)

In [ ]:
img_matches = cv2.drawMatches(img1, kp1, img2, kp2, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
t2=time.time();
time_sum+=t2-t1;

In [ ]:
plt.figure(figsize=(10, 5))
plt.imshow(img_matches)
plt.title("Good Matches with Ratio Test")
plt.show()

In [ ]:
t1=time.time();
print('final')

In [ ]:
if len(good_matches) > 4:

    src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    #print(len(good_matches));
    #print(src_pts.shape);

    H, mask = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 3 )
    





    
    img1=img1_cpy
    img2=img2_cpy
    h1, w1 = img1.shape[:2]
    h2, w2 = img2.shape[:2]



    result1 = cv2.warpPerspective(img2, H, (w1 + w2, h2))
    result = np.zeros((h1, w1 + w2, 3), dtype=np.uint8)
    _, mmask = cv2.threshold(result1, 1, 255, cv2.THRESH_BINARY_INV)

    result2=cv2.bitwise_and(img1[0:h1,0:w1], mmask[0:h1,0:w1])


    result[0:h1, 0:w1] = result2
    result=cv2.bitwise_or(result,result1);
    


    t2=time.time();
    time_sum+=t2-t1;
    print('over')
    # ???????
    cv2.imwrite('89.jpg',result);
    plt.figure(figsize=(10, 5))
    plt.imshow(result, cmap='gray')
    plt.title("Image Stitching Result")
    plt.show()
else:
    print("no matrix")
    exit();

In [ ]:
elapsed_time = time_sum;
fps = 1 / elapsed_time if elapsed_time > 0 else 0

In [ ]:
print("fps:",fps);
print("time_cost(second):",time_sum);